In [175]:
import collections
from itertools import permutations, combinations, combinations_with_replacement, product
from scipy.special import comb, binom
from math import factorial
import random
import numpy as np
import pandas as pd
import time

In [9]:
# Normal deck
ranks = [str(x) for x in range(2,11)] + list('JQKA')
suits = list('HDCS')
deck = list(product(ranks, suits))
values = dict(zip(range(1, 15),['A']+ranks))

In [3]:
# Pythonic deck
Card = collections.namedtuple('Card', ['rank', 'suit'])
py_deck = [Card(rank, suit) for rank in ranks for suit in suits]

In [32]:
# generating a random draw of five cards - "A Poker Hand"
hand = random.sample(py_deck,k=5) # unordered selection of 5 cards without repacement

In [47]:
# Counting the number of ranks and suits in a given hand
def get_counters(hand):
    c_ranks = collections.Counter([a[0] for a in hand])
    c_suits = collections.Counter([a[1] for a in hand])
    c_values = collections.Counter([key for key,val in values.items() if val in c_ranks])
    c_values_noA = collections.Counter([key for key,val in values.items() if (val in c_ranks) & (val!='A')])
    return _c_ranks, _c_suits, _c_values, _c_values_noA

#### Order of Poker hands:
1. Royal Flush: A, K, Q, J, 10 all same suit
2. Straight Flush: 5 consecutive cards all same suit
3. Four of a Kind: 4 cards of same rank
4. Full House: 3 cards of same rank + 2 another of same rank
5. Flush: Any five cards of same suit but not in a sequence
6. Straight: Five cards in a sequence, not of same suit
7. Three of a kind: 3 cards of same rank
8. Two pair: 2 cards of same rank + 2 cards of same rank
9. One pair: 2 cards of same rank

In [63]:
## Key learnign whendefining the condition for straigh:
# Incorrect: (((_A_in_ranks)&(_val_range_noA==3))|((~_A_in_ranks)&(_val_range_wA==4)))
# Just the range being 4 will not work as there could be 2,2,2,2,6 will also satify the range condition



In [209]:
def evaluator(hand):
    _c_ranks, _c_suits, _c_values, _c_values_noA = get_counters(hand)
    ## Individual conditions
    _royal_condn = (_c_ranks == collections.Counter(['A', 'K', 'Q', 'J', '10']))
    _flush_condn = (_c_suits.most_common(1)[0][1] == 5)
    _A_in_ranks = ('A' in _c_ranks)
#     _val_range_wA = (max(_c_values.keys())-min(_c_values.keys()))
#     _count_vals_wA = len(_c_values.keys())
    _val_range_noA = (max(_c_values_noA.keys())-min(_c_values_noA.keys()))
    _count_vals_noA = len(_c_values_noA.keys())
    _straight_condn = (
                        ((_A_in_ranks)&(_val_range_noA==3)&(_count_vals_noA==4))|
                        ((~_A_in_ranks)&(_val_range_noA==4)&(_count_vals_noA==5))
                      )
    _most_common_rank1, _most_common_rank2 = _c_ranks.most_common(1)[0][1], _c_ranks.most_common(2)[1][1]
    _rank1_gt4, _rank1_gt3, _rank1_gt2 = (_most_common_rank1 >= 4), (_most_common_rank1 >= 3), (_most_common_rank1 >= 2)
    _rank2_gt2 = (_most_common_rank2 >= 2)
    ## Special Hand Conditions
    _special_hands = collections.OrderedDict([('royal_flush', (_royal_condn&_flush_condn))
                                         , ('straight_flush', (_flush_condn&_straight_condn))
                                         , ('four_kind', _rank1_gt4)
                                         , ('full_house', (_rank1_gt3&_rank2_gt2))
                                         , ('flush', (_flush_condn&~_straight_condn))
                                         , ('straight', (_straight_condn&~_flush_condn))
                                         , ('three_kind', _rank1_gt3)
                                         , ('two_pair', (_rank1_gt2&_rank2_gt2))
                                         , ('one_pair', _rank1_gt2)
                                         ,   
                                        ])
    for _ in _special_hands:
        if _special_hands[_]:
            return _
    else:
        return 'not_special'

In [210]:
def cut_hands(deck=py_deck, num_hands=5):
    hands = []
    for i in range(num_hands):
        hand = random.sample(py_deck,k=5)
        hands.append(hand)
    return hands

In [211]:
def special_hand_counter(hands, prob=False):
    _accum = []
    for hand in hands:
        val = evaluator(hand)
        _accum.append(val)
    counts = collections.Counter(_accum)
    total = sum(counts.values())
    if prob:
        d={}
        for k, v in counts.items():
            d[k] = v/total
        return d
    else:
        return counts

In [230]:
# Compute probabilities of eah of the above hands through simulation
# Verify with analytically computed probabilities where possible

## Running the n simulation
    # Each round of simulation draws 1Mn hands from the deck
hands_index = ['royal_flush', 'straight_flush', 'four_kind', 'full_house'
               , 'flush', 'straight', 'three_kind', 'two_pair', 'one_pair', 'not_special']
df = pd.DataFrame(hands_index, columns=['hand_ranks'])
start_time = time.time()
n = 10
n_hands = int(1e6)
for i in range(n):
    sim_num = 'Sim_'+str(i)
    print(f'Starting {sim_num} run now.')
    sim_start_time = time.time()
    H = cut_hands(num_hands=n_hands)
    sp_count = special_hand_counter(H, prob=True)
    sp_count = pd.DataFrame({'hand_ranks':sp_count.keys(), sim_num:sp_count.values()})
    df = df.merge(sp_count, on='hand_ranks', how='left')
    sim_end_time = time.time()
    print(f'{sim_num} ended. This run took {sim_end_time-sim_start_time} seconds to complete')
df['avg_probs'] = df.mean(axis=1)
df['std_err_probs'] = np.sqrt(df['avg_probs']*(1-df['avg_probs'])/(n*num_hands))
df['avg_probs_lower'] = df['avg_probs']-2*df['std_err_probs']
df['avg_probs_upper'] = df['avg_probs']+2*df['std_err_probs']


end_time = time.time()
print(f'{n} simulations took a total of {end_time-start_time} seconds')

Starting Sim_0 run now
Sim_0 ended. This run took 42.6960723400116 seconds to complete
Starting Sim_1 run now
Sim_1 ended. This run took 44.413750648498535 seconds to complete
Starting Sim_2 run now
Sim_2 ended. This run took 44.95330786705017 seconds to complete
Starting Sim_3 run now
Sim_3 ended. This run took 44.6486759185791 seconds to complete
Starting Sim_4 run now
Sim_4 ended. This run took 46.586398124694824 seconds to complete
Starting Sim_5 run now
Sim_5 ended. This run took 46.018006801605225 seconds to complete
Starting Sim_6 run now
Sim_6 ended. This run took 46.12770080566406 seconds to complete
Starting Sim_7 run now
Sim_7 ended. This run took 44.889774560928345 seconds to complete
Starting Sim_8 run now
Sim_8 ended. This run took 45.006590843200684 seconds to complete
Starting Sim_9 run now
Sim_9 ended. This run took 44.751603841781616 seconds to complete
Simulations took a total of 450.09888315200806 seconds


C:\Users\sande\AppData\Local\Temp/ipykernel_14264/4272466263.py:22: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['avg_probs'] = df.mean(axis=1)


In [231]:
## Analytically compute probabilities of each special hand

## Total number of possible combinations
total_comb = comb(52, 5)

## Probability of royal_flush
# Royal Flush: A, K, Q, J, 10 all same suit

# of suits to choose for the royal straigh range = comb(4,1)

# total nunmber of favorable of outcomes = comb(4,1)

p_royal_flush = comb(4,1)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
## Probability of straight_flush
# Straight Flush: 5 consecutive cards all same suit

# of suits to choose for the straight range = 4C1 = comb(4, 1)
# for each suit we need to compute the # of possible straight sequences
# A can double up as both value=1 and value=14
## First straight sequence = A, 2, 3, 4, 5
## Last straight sequence = 10, J, Q, K, A
## Total 14 cards in full range, and there is one and only one straight range possible starting with each of the cards in
## the sequence A,2,3,...,10 - Total 10 sequences

# total nunmber of favorable of outcomes = comb(4,1)*10

# total possibilities = total comb
p_straight_flush = comb(4,1)*10/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of four_kind
# Four of a Kind: 4 cards of same rank

# of ranks to pick from = comb(13,1)
# of cards to pick from a particular suit = comb(4,4)
# of possibilities for the fifth card = 52-4 = 48

# total nunmber of favorable of outcomes = comb(13,1)*comb(4,4)*48

p_four_kind = comb(13,1)*comb(4,4)*48/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of full_house
# Full House: 3 cards of same rank + 2 cards of same rank

## Need to have 3 cards of a kind and 2 cards of a different kind

## of ways to select the rank having 3 cards = comb(13,1)
## Of this particular rank, number of ways to choose three cards from the available 4 suits = comb(4,3)
## of ways to select the rank having 2 cards from the remaining ranks = comb(12,1)
## Of this particular rank, number of ways to choose two cards from the available 4 suits = comb(4,2)

# total nunmber of favorable of outcomes = comb(13,1)*comb(4,3)*comb(12,1)*comb(4,2)

p_full_house = comb(13,1)*comb(4,3)*comb(12,1)*comb(4,2)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of flush
# Flush: Any five cards of same suit but not in a sequence

# Number of ways to choose a suit from available = comb(4,1)
# Number of ways to choose 5 cards from ranks available from the chosen suit = comb(13, 5)
# Number of combinations having a straigh sequence from this list of sequences = 10
# Number of hands withoout a straigh sequence = (comb(13, 5)-10)

# total nunmber of favorable of outcomes = comb(13, 5)*(comb(13, 5)-10)

p_flush = comb(13,1)*comb(4,3)*comb(12,1)*comb(4,2)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of straight
# Straight: Five cards in a sequence, not of same suit

# A total number of 10 straight sequences are possible each starting with A, 2, ...., 10
# For each of these sequences, there are a total of 4**5 combinations of suits possible
# However, one of the combinations would be a straigh flush, i.e. all cards from the same suit
# There are 4 such combinations possible, 1 for each suit
# Hence total number of suit combinations for each sequence excluding the flush is 4**5-4

# total nunmber of favorable of outcomes = 10*(4**5-4)

p_straight = 10*(4**5-4)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of three_kind
# Three of a kind: 3 cards of same rank

# Number of ways to choose a rank for the 3 cards = comb(13,1)
# Number of ways to choose 3 cards from 4 for the selected rank = comb(4,3)
# Number of ways to choose 2 cards from the remaining ranks = comb(12*4,2)

# total nunmber of favorable of outcomes = comb(13,1)*comb(4,3)*comb(12*4,2)

p_three_kind = comb(13,1)*comb(4,3)*comb(12*4,2)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of two_pair
# Two pair: 2 cards of same rank + 2 cards of another rank

# Number of ways to choose 2 rank for the 4 cards = comb(13,2)
# Number of ways to choose 2 cards from 4 for the selected rank = comb(4,2)
# Number of ways to choose 1 cards from the remaining ranks = comb(11*4,1)

# total nunmber of favorable of outcomes = comb(13,2)*comb(4,2)*comb(4,2)*comb(11*4,1)

p_two_pair = comb(13,2)*comb(4,2)*comb(4,2)*comb(11*4,1)/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of one_pair
# One pair: 2 cards of same rank

# Number of ways to choose 1 rank for the 2 cards = comb(13,1)
# Number of ways to choose 2 cards from 4 for the selected rank = comb(4,2)
# Number of ways to choose 3 cards from the remaining ranks such that they do not contain another pair = (12*4)*(11*4)*(10*4)
# The other three cards cam be permuted in 3! ways amongst themselves

# total nunmber of favorable of outcomes = comb(13,1)*comb(4,2)*((12*4)*(11*4)*(10*4)/factorial(3))

p_one_pair = (comb(13,1)*comb(4,2)*((12*4)*(11*4)*(10*4)/factorial(3)))/comb(52, 5)

#-----------------------------------------------------------------------------------------------------------------------------
# Probability of not_special
# Note special: none of the above

p_not_special = 1-sum([p_royal_flush, p_straight_flush, p_four_kind, p_full_house
                     , p_flush, p_straight, p_three_kind, p_two_pair, p_one_pair])

#-----------------------------------------------------------------------------------------------------------------------------
# Hands index probabilities

hands_index_ana_probs = [p_royal_flush, p_straight_flush, p_four_kind, p_full_house
                     , p_flush, p_straight, p_three_kind, p_two_pair, p_one_pair, p_not_special]

In [232]:
df_ana = pd.DataFrame(zip(hands_index, hands_index_ana_probs), columns=['hand_ranks', 'analytical_probs'])
df_ana    # probability of getting a royal flush is 2 in a million, hence number of draws per simulation has to be atleast 1Mn 

,hand_ranks,analytical_probs
0,royal_flush,0.000002
1,straight_flush,0.000015
2,four_kind,0.000240
3,full_house,0.001441
4,flush,0.001441
5,straight,0.003925
6,three_kind,0.022569
7,two_pair,0.047539
8,one_pair,0.422569
9,not_special,0.500260


In [233]:
df = df.merge(df_ana, on='hand_ranks', how='left')
df['mu_in_CI'] = ((df.analytical_probs>=df.avg_probs_lower)|(df.analytical_probs<=df.avg_probs_upper))

In [234]:
df

,hand_ranks,Sim_0,Sim_1,Sim_2,Sim_3,Sim_4,Sim_5,Sim_6,Sim_7,Sim_8,Sim_9,avg_probs,std_err_probs,avg_probs_lower,avg_probs_upper,analytical_probs,mu_in_CI
0,royal_flush,NaN,NaN,0.000001,0.000002,0.000003,0.000002,0.000004,0.000003,NaN,0.000001,0.000002,4.780909e-07,0.000001,0.000003,0.000002,True
1,straight_flush,0.000018,0.000025,0.000021,0.000032,0.000022,0.000020,0.000020,0.000029,0.000028,0.000023,0.000024,1.542707e-06,0.000021,0.000027,0.000015,True
2,four_kind,0.000240,0.000253,0.000271,0.000198,0.000240,0.000251,0.000249,0.000246,0.000234,0.000245,0.000243,4.925861e-06,0.000233,0.000253,0.000240,True
3,full_house,0.001508,0.001535,0.001415,0.001436,0.001402,0.001472,0.001370,0.001459,0.001437,0.001512,0.001455,1.205190e-05,0.001430,0.001479,0.001441,True
4,flush,0.001984,0.001915,0.001924,0.001962,0.001953,0.002000,0.001950,0.001988,0.002016,0.001950,0.001964,1.400122e-05,0.001936,0.001992,0.001441,True
5,straight,0.006739,0.006566,0.006565,0.006702,0.006840,0.006586,0.006704,0.006625,0.006595,0.006571,0.006649,2.570036e-05,0.006598,0.006701,0.003925,True
6,three_kind,0.021298,0.021277,0.021168,0.021015,0.020913,0.021115,0.021163,0.021055,0.021110,0.020943,0.021106,4.545355e-05,0.021015,0.021197,0.022569,True
7,two_pair,0.047534,0.047519,0.047552,0.047338,0.047592,0.047587,0.047544,0.047508,0.047734,0.047536,0.047544,6.729334e-05,0.047410,0.047679,0.047539,True
8,one_pair,0.421613,0.423084,0.422853,0.422189,0.422943,0.422872,0.421955,0.422932,0.422149,0.422005,0.422459,1.562010e-04,0.422147,0.422772,0.422569,True
9,not_special,0.499066,0.497826,0.498230,0.499126,0.498092,0.498095,0.499041,0.498155,0.498697,0.499214,0.498554,1.581132e-04,0.498238,0.498870,0.500260,True


In [ ]:
# Create an Evaluator class to do the checking
# Draw back if that it may take up too much space and may be slower than what we need